In [1]:
from utils import back_translate
from utils import compute_bleu_chrf
path = "/home/mshahidul/project1/data2/extracted_files/eng_spa_pairs/medlineplus_pairs.txt"
f = open(path, "r").read().split("\n")

In [4]:

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
                    ### Instruction:
                    {}

                    ### Input:
                    {}

                    ### Response:
                    {}"""
max_seq_length=2048
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "/home/mshahidul/project1/model/unsloth/DeepSeek-R1-Distill-Qwen-7B", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = None,
        load_in_4bit = False,
    )
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla V100-DGXS-32GB. Max memory: 31.727 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1. CUDA: 7.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 4/4 [00:06<00:00,  1.52s/it]
Unsloth 2024.12.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
ans_cal=[]

In [6]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

def inference(ques):
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            "Translate the input into English to Spanish:", # instruction
            ques, # input
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
    ans=tokenizer.batch_decode(outputs)
    start_marker = '### Response:\n'
    end_marker = '<|end_of_text|>'

    # Find the start and end positions
    start_index = ans[0].find(start_marker) + len(start_marker)
    end_index = ans[0].find(end_marker)
    response = ans[0][start_index:end_index].strip()
    return response 

# Extract the response
import tqdm
for x in tqdm.tqdm(f[:100]):
    xx=x.split("\t")
    hyp=inference(xx[0])
    back_eng=back_translate(hyp)
    score=compute_bleu_chrf(xx[0], back_eng)
    ans_cal.append({
        "original_eng":xx[0],
        "original_spa":xx[1],
        "tran_spa":hyp,
        "back_eng":back_eng,
        "bleu_score":score['bleu_score'],
        "chrf_score":score['chrF++']
    })

100%|██████████| 100/100 [03:40<00:00,  2.21s/it]


In [7]:
avg=0
avg_chrf=0
for x in ans_cal:
    avg+=x['bleu_score']
    avg_chrf+=x['chrf_score']
avg/=len(ans_cal)
avg_chrf/=len(ans_cal)
print(f"bleu score: {avg}, chrf++: {avg_chrf}")

bleu score: 36.40244991412947, chrf++: 37.726370852652074
